In [1]:
import os
import numpy as np
# import working directory to check functions
os.chdir('/Users/Dominik/R-workspace/cecelia/inst')

# MacOS
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# config
import py.config_utils as cfg

In [2]:
# how to import CODEX data that is registered but not assembled?
im_path = '/Users/Dominik/Downloads/21-013_SP_CC2-C no LED 07DEC2021_reg001.tif'

In [102]:
im_path = '/Volumes/MDHS-Research/5300/5260/Lab-Mueller/Dominik Schienstock/Notebook/Experiments/HuBMAP/SPLEEN/HBM946.WMTC.283.ome.tif'

In [103]:
import tifffile
im = tifffile.TiffFile(im_path)

In [104]:
im.ome_metadata

'<?xml version="1.0" encoding="UTF-8"?><OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" UUID="urn:uuid:867d0aed-0043-4a7e-8c50-71e812d9c324" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd"><Image ID="Image:0" Name="default.png"><AcquisitionDate>2021-11-21T20:45:59.394</AcquisitionDate><Pixels BigEndian="true" DimensionOrder="XYZCT" ID="Pixels:0" Interleaved="false" PhysicalSizeX="377.4" PhysicalSizeXUnit="nm" PhysicalSizeY="377.4" PhysicalSizeYUnit="nm" SignificantBits="16" SizeC="11" SizeT="1" SizeX="9974" SizeY="9490" SizeZ="1" Type="uint16"><Channel ID="Channel:0:0" Name="CD107a" SamplesPerPixel="1"><LightPath/></Channel><Channel ID="Channel:0:1" Name="CD11c" SamplesPerPixel="1"><LightPath/></Channel><Channel ID="Channel:0:2" Name="CD20" SamplesPerPixel="1"><LightPath/></Channel><Channel ID="Channel:0:3" Name="CD21" SamplesPerPixel="1">

In [5]:
import py.ome_xml_utils as ome_xml_utils
from py.dim_utils import DimUtils

x_array = im.asarray()

In [32]:
im_meta = im.imagej_metadata

In [16]:
new_shape = list(x_array.shape)
new_shape = tuple([new_shape[0] * new_shape[1]] + new_shape[2:])

In [45]:
# reshuffle array
idx_rm = [i for i, x in enumerate(im_meta['Labels']) if x != 'DAPI-02' and x.startswith(('DAPI-', 'Blank', 'Empty'))]
y_array = np.delete(x_array.reshape(new_shape), idx_rm, axis = 0)

In [51]:
zarr_path = '/Users/Dominik/Downloads/21-013_SP_CC2-C no LED 07DEC2021_reg001.zarr'

In [71]:
import py.zarr_utils as zarr_utils

nscales = 6

im_chunks = [1] * len(y_array.shape)
im_chunks[-2] = 2048
im_chunks[-1] = 2048
im_chunks

zarr_utils.create_multiscales(y_array, zarr_path, nscales = nscales,
                              im_chunks = im_chunks, x_idx = 2, y_idx = 1)

In [89]:
im_channels = [x for x in im_meta['Labels'] if x == 'DAPI-02' or not x.startswith(('DAPI-', 'Blank', 'Empty'))]

In [92]:
from ome_types import model

physical_size_x = 0.0003774

# create metadata
# TODO anything else?
pixels = model.Pixels(
    size_c = y_array.shape[0],
    size_x = y_array.shape[2],
    size_y = y_array.shape[1],
    size_t = 1,
    size_z = 1,
    physical_size_x = physical_size_x,
    physical_size_y = physical_size_x,
    physical_size_x_unit = im_meta['unit'],
    physical_size_y_unit = im_meta['unit'],
    type = 'uint16',
    dimension_order = model.Pixels_DimensionOrder.XYZCT,
    metadata_only = True
)

for x in im_channels:
    pixels.channels.append(model.Channel(name = x))

# add metadata
im_metadata = model.OME(images=[model.Image(pixels=pixels)])

ome_xml_utils.write_ome_xml(zarr_path, im_metadata)

In [95]:
from py.napari_utils import NapariUtils

In [97]:
napari_utils = NapariUtils()

In [98]:
napari_utils.open_viewer()

In [101]:
napari_utils.open_image(zarr_path, use_channel_axis = True, channel_names = im_channels, visible = False)

<zarr.hierarchy.Group '/' read-only>
[34, 9072, 9408]
[9408, 9072, 34]
['X', 'Y', 'C']
[34, 9072, 9408]


In [ ]:
im, _ = zarr_utils.open_as_zarr(im_path, as_dask=True)

In [ ]:
import napari

viewer = napari.viewer.Viewer()

In [ ]:
viewer.add_image(im, channel_axis=1, visible=False)

In [ ]:
import skimage.filters
import skimage.morphology

In [ ]:
avg_slices = [slice(None) for _ in range(len(im.shape))]
avg_slices[x_idx] = slice(500, 1000, 1)
avg_slices[y_idx] = slice(2000, 2500, 1)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(10,c_val * 5))
percentile = 75

for i, x in enumerate(range(c_val)):  
    slices[c_idx] = x
    plt.subplot(c_val,2,(i*2)+1)
    plt.imshow(np.squeeze(im[tuple(slices)]), cmap='gray')
    plt.title(i)
    
    plt.subplot(c_val,2,(i*2)+2)
    plt.imshow(np.squeeze(
        (np.squeeze(im[tuple(slices)]) + 1) / (np.percentile(im[tuple(avg_slices)].compute(), percentile, axis = c_idx).astype(np.uint8) + 1)
    ), cmap='magma')
    plt.title(f'DIFF {i}')